In [1]:
import numpy as np
from IPython.core.debugger import set_trace
import re
import difflib
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import time
import random
import pickle
from lxml.html import fromstring
from collections import Counter
from requests.packages.urllib3.util import Retry
from requests.adapters import HTTPAdapter
from requests import Session, exceptions
import sys
import pymongo
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import os
import json
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
from nltk.tokenize import sent_tokenize, word_tokenize
sys.path.append("/Users/rohanramesh/Documents/GitHub/Insight_writers/Dash_to_server/")
from text_processing import ProcessArticle as pa
import suggestions as s
import difflib
from scipy.stats import wilcoxon, sem

# Grab Podcast information
The goal of this is to validate my writer style recommendations by showing that writers with a similar writing style are more likely to do a podcast together than chance. To do this I will use the listennotes api to get the metdata about the 50 most recent podcasts for each writer and check to see if the last name of another author was in the same podcast.

In [2]:
# data load
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/Attempt3_mr_scrape.pickle', 'rb') as handle:
    scrapevar = pickle.load(handle)

# load writer df
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/writer_df.pickle', 'rb') as handle:
    writer_df = pickle.load(handle)

# load writer features
writer_features = pd.read_pickle('/Users/rohanramesh/Documents/Insight/data_bball_writers/writer_features_USE.pickle')

# load podcasts - IF ALREADY SCRAPED
top_podcasts = pd.read_pickle('/Users/rohanramesh/Documents/Insight/data_bball_writers/top50podcasts.pickle')

In [8]:
# grab config file that has the apikeys for listennotes
# keys[0] apple, keys[1] listen notes
path_teams = '/Users/rohanramesh/Documents/Insight/data_bball_writers/config.txt'
with open(path_teams, 'r') as f:
    tmp = f.readlines()
keys = [i.rstrip() for i in tmp]

Pull necessary info from writer and article database

In [3]:
# grab the relevant information from the database of writers and articles
dbname = 'writer_feature_db'
# dbname = 'writer_db'
username = 'rohanramesh' # change this to your username
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))
con = None
con = psycopg2.connect(database = dbname, user = username)

postgres://rohanramesh@localhost/writer_feature_db
True


In [4]:
# query to load entire db into pandas
sql_query = """
SELECT * FROM writer_feature;
"""
writer_feature_df = pd.read_sql_query(sql_query,con)

In [5]:
sql_query = """
SELECT AVG("n_words") AS "avg_n_words",
         AVG("neg_sent") AS "avg_neg_sent",
         AVG("neu_sent") AS "avg_neu_sent",
         AVG("pos_sent") AS "avg_pos_sent",
         AVG("neg_sent_var") AS "avg_neg_sent_var",
         AVG("neu_sent_var") AS "avg_neu_sent_var",
         AVG("pos_sent_var") AS "avg_pos_sent_var",
        AVG("n_sentences") AS "avg_n_sentences",
        AVG("n_wordspersentence") AS "avg_n_wordspersentence",
        AVG("n_wordspersent_variability") AS "avg_n_wordspersent_variability",
        AVG("wordlength") AS "avg_wordlength",
        AVG("wordlength_var") AS "avg_wordlength_var",
        AVG("wordlength_skew") AS "avg_wordlength_skew",
        "author_list"
FROM writer_feature
WHERE "n_words" > 100
GROUP BY "author_list"
HAVING COUNT(*) > 25 
ORDER BY AVG("n_words") DESC
"""
writer_feature_subsection = pd.read_sql_query(sql_query,con)
writer_feature_subsection.head(10)
writer_feature_subsection.replace([np.inf, -np.inf], np.nan)
writer_feature_subsection = writer_feature_subsection.dropna(axis=0, how='any')

In [7]:
# Build an author list with full names not just website label names
author_webnames = writer_feature_subsection['author_list']
author_list = []
for i in author_webnames:
    idx = writer_df['website_name'] == i
    author_list.append(writer_df['Idea Text'][np.where(idx)[0]].tolist()[0])

Here is where I actually use api to grab top 50 podcasts

In [ ]:
# actually grab podcast information using the listen notes api
top_podcasts = {}
# iterate through all authors
for curr_author in author_list:
    top_podcasts[curr_author] = []
    # break up into first and last name for formatting on their website purposes
    r = word_tokenize(curr_author)
    # get 10 hits per page so grab the top 50
    page_idx = 0
    while page_idx < 50:
        # url in their formatting, searching for a given author, not sorting by data, sorting by relevance
        url = ("https://listennotes.p.mashape.com/api"
            "/v1/search?offset={}&q={}+{}&sort_by_date=0&type=episode".format(page_idx,r[0], r[1]))
        response = requests.get(url,
          headers={
            "X-Mashape-Key": keys[1],
            "Accept": "application/json"
          }
        )
        a = response.content
        b = json.loads(a)
        top_podcasts[curr_author].append(b)
        page_idx += 10
        time.sleep(random.uniform(0.5,2))
        print(url)


In [9]:
# for each podcast search through all descriptions for other authors name and if their is a comention
# then set that match to 1 in the author mentions matrix (authors x authors)
author_mentions = np.zeros([len(author_list), len(author_list)])
for i in range(0,len(author_list)):
    curr_author = author_list[i]
    # only have 4 pages bc going through top 50 podcasts
    for curr_page in range(0,4):
        # have up to 10 hits per page and searching in the results section
        for curr_pod in range(0,np.min([9,len(top_podcasts[curr_author][curr_page]['results'])])):
            # this is the info from this podcast
            curr_info = top_podcasts[curr_author][curr_page]['results'][curr_pod]
            podcast_keys = curr_info.keys()
            matches = []
            # iterate through all metadata information and search for every possible last name
            for curr_key in podcast_keys:
                for j in range(0,len(author_list)):
                    curr_authorcomp = author_list[j]
                    # bc of silly formatting for Jason Concepcion
                    if curr_authorcomp == 'Jason Concepcion/netw3rk':
                        curr_authorcomp = 'Jason Concepcion'
                    search_terms = curr_authorcomp
                    if isinstance(curr_info[curr_key], str):
                        last_name = word_tokenize(search_terms)[1]
                        # if last name of other author is in the metadata then set
                        # author_mentions[author1,author2] = 1 
                        # so can find co-mentions later
                        if last_name in curr_info[curr_key]:
                            author_mentions[i,j] = 1
                        

For writer co-mentions I don't care if Zach Lowe is on Lee Jenkins' podcast or vice versa so will take a mention in either direction as evidence they have done a podcast together

In [67]:
# iterate each author and see how many times match in podcasts in either direction
website_author_names = writer_df['website_name'].tolist()
matched_fraction = []
nonmatch_fraction = []
for ii in range(0,len(author_list)):
    # if writer has one or fewer hits (ie. doesn't have a podcast) then ignore
    if np.sum(author_mentions[ii,:]) <= 1:
        print(author_list[ii])
        continue
    curr_author = author_list[ii]
    website_name = s.match_author_names(writer_df, curr_author)
    # this is the writers that are suggested based off the writing style of current writer
    author_sugg = s.give_author_suggestion_from_author(writer_features, curr_author)
    author_sugg_list = author_sugg['Author_wn'].tolist()
    matched_authors = author_sugg_list[0:3] # already removed same author - looking at top 3 hits
    # these are all of the nonmatched authors
    nonmatched_authors = [i for i in website_author_names if (i not in matched_authors)
                         and (i != curr_author)]
    # get n and fraction for matched authors - ie how many out of top were on a podcast together
    n_match = []
    for i in matched_authors:
        idx1 = author_list.index(curr_author)
        idx2 = author_list.index(match_author_names(writer_df, i))
        val = np.max([author_mentions[idx1,idx2], author_mentions[idx2,idx1]])
        n_match.append(val)
    matched_fraction.append(np.sum(n_match)/len(n_match))
    # get n and fraction for nonmatched
    n_nonmatch = []
    for i in nonmatched_authors:
        if match_author_names(writer_df, i) in author_list:
            idx1 = author_list.index(curr_author)  
            idx2 = author_list.index(match_author_names(writer_df, i))
            val = np.max([author_mentions[idx1,idx2], author_mentions[idx2,idx1]])
            n_nonmatch.append(val)
    nonmatch_fraction.append(np.sum(n_nonmatch)/len(n_nonmatch))


Chris Ballard
Justin Jett


In [1]:
# # plot distributions, look at means, do rank-sum test
plt.hist(nonmatch_fraction)
plt.show()
plt.hist(matched_fraction)
plt.show()
print(np.mean(matched_fraction))
print(np.mean(nonmatch_fraction))
print(np.mean(matched_fraction)/np.mean(nonmatch_fraction))
a,pval = wilcoxon(matched_fraction, nonmatch_fraction)
pval
# plot barplot for matched vs random
plt.bar(range(0,2),[np.mean(matched_fraction)*100, np.mean(nonmatch_fraction)*100], 0.75, 
        yerr=[sem(matched_fraction)*100, sem(nonmatch_fraction)*100], tick_label= ['Suggested', 'Random'])
plt.ylabel('Percent of writers to do podcasts together')
plt.savefig('/Users/rohanramesh/Documents/Insight/Presentation_material/Figures/Podcast.eps', format='eps', dpi=1000)
plt.show()
# pval